In [1]:
import pandas as pd
import random
import ast

In [2]:
data = pd.read_csv('notes_cleaned.csv')
data['target'] = data['target'].apply(lambda x: ast.literal_eval(x))

In [3]:
data

,summary,target
0,forbes has a good rundown of the investigation...,"[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
1,they are expressing a personal opinion in a st...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
2,teslas purchased after are not eligible for u...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
3,the jan th riots were encouraged by the sittin...,"[0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]"
4,the committee has been found by numerous court...,"[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
...,...,...
62475,around of the consultants employed by nhs eng...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
62476,the poster alleges the only strategy currently...,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
62477,the ads elon is seeing can be stopped by eithe...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
62478,jason has a personal interest in this,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [4]:
import ast

pairs = []

for _ in range(16000):
    # Select two random indices
    idx1, idx2 = random.sample(range(len(data)), 2)
    
    # Get summaries and targets for the selected indices
    summary1, summary2 = data.loc[idx1, 'summary'], data.loc[idx2, 'summary']
    target1, target2 = data.loc[idx1, 'target'], data.loc[idx2, 'target']
    
    # Determine similarity based on target
    similar = [1 if target1[i] == 1 and target2[i] == 1 else 0 for i in range(len(target1))]
    
    # Add pair to the list
    pairs.append((summary1, summary2, similar))

# Convert pairs list to DataFrame
pairs_data = pd.DataFrame(pairs, columns=['sentence1', 'sentence2', 'similar'])

In [5]:
pairs_data

,sentence1,sentence2,similar
0,the photos clearly show that a megaphone was p...,the unsealed documents in the depp vs heard de...,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
1,while not aligned with mainstream accepted nar...,this lacks context the town in question is not...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,president magill announced an action plan to c...,post is just a question to the audience anyon...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,its correct according to us government source...,these statements are false claims and are unsu...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,op is expressing their personal story and out...,a common misconception is that ar is an acrony...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
15995,bls stats states that unemployment was in feb...,elon has a shrimp dick,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
15996,as of the date of this the us domestic price ...,fauci an immunologist is the presidents chief ...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
15997,bernie sanders got me arrested refers to the s...,stew peters is irresponsible and misleading it...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
15998,ivermectin has shown no benefit for the preven...,the process is easy not convoluted the top fr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
EMBEDDING_DIM = 50

MAX_SEQUENCE_LENGTH = 10
VALIDATION_SPLIT = 0.1


RATE_DROP_LSTM = 0.17
RATE_DROP_DENSE = 0.25
NUMBER_LSTM = 50
NUMBER_DENSE_UNITS = 50
ACTIVATION_FUNCTION = 'relu'


siamese_config = {
	'EMBEDDING_DIM': EMBEDDING_DIM,
	'MAX_SEQUENCE_LENGTH' : MAX_SEQUENCE_LENGTH,
	'VALIDATION_SPLIT': VALIDATION_SPLIT,
	'RATE_DROP_LSTM': RATE_DROP_LSTM,
	'RATE_DROP_DENSE': RATE_DROP_DENSE,
	'NUMBER_LSTM': NUMBER_LSTM,
	'NUMBER_DENSE_UNITS': NUMBER_DENSE_UNITS,
	'ACTIVATION_FUNCTION': ACTIVATION_FUNCTION
}

In [7]:
from handler import word_embed_meta_data, create_test_data

In [8]:
sentences1 = list(pairs_data['sentence1'])
sentences2 = list(pairs_data['sentence2'])
similar = list(pairs_data['similar'])

In [9]:
tokenizer, embedding_matrix = word_embed_meta_data(sentences1 + sentences2,  siamese_config['EMBEDDING_DIM'])

embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}

## creating sentence pairs
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]

Embedding matrix shape: (32689, 50)
Null word embeddings: 1


In [10]:
from model import SiameseBiLSTM

In [ ]:
class Configuration(object):
    """Dump stuff here"""

CONFIG = Configuration()
CONFIG.embedding_dim = siamese_config['EMBEDDING_DIM']
CONFIG.max_sequence_length = siamese_config['MAX_SEQUENCE_LENGTH']
CONFIG.number_lstm_units = siamese_config['NUMBER_LSTM']
CONFIG.rate_drop_lstm = siamese_config['RATE_DROP_LSTM']
CONFIG.number_dense_units = siamese_config['NUMBER_DENSE_UNITS']
CONFIG.activation_function = siamese_config['ACTIVATION_FUNCTION']
CONFIG.rate_drop_dense = siamese_config['RATE_DROP_DENSE']
CONFIG.validation_split_ratio = siamese_config['VALIDATION_SPLIT']

siamese = SiameseBiLSTM(CONFIG.embedding_dim , CONFIG.max_sequence_length, CONFIG.number_lstm_units , CONFIG.number_dense_units, 
					    CONFIG.rate_drop_lstm, CONFIG.rate_drop_dense, CONFIG.activation_function, CONFIG.validation_split_ratio)

best_model_path = siamese.train_model(sentences_pair, similar, embedding_meta_data, model_save_directory='./')

In [ ]:
from keras.models import load_model
from operator import itemgetter

In [ ]:
model = load_model(best_model_path)